In [83]:
import pandas as pd
import numpy as np
import string

In [84]:
data_train = pd.read_pickle("Data/train_data.pkl")

In [85]:
data_train.head()

,comment_text,toxic,text_clean
22689,"""::No. The reason some excellent editors leav...",0,reason excellent editor leave reasoned fully e...
154371,Comment Will it merge back into Historicity of...,0,comment merge back historicity jesus tiny part...
93418,Well then I might do that if I don't get the l...,0,well might dont get list done time although st...
31049,"Several years later, I finally add something t...",0,several year later finally add something talk ...
47906,Could you please specify the licence of the ph...,0,could please specify licence photo download se...


In [86]:
def preprocess_text(text, remove_stop = True, stem_words = False, remove_mentions_hashtags = True):
    text = text.lower()
    text = text.replace("what's", "what is")
    text = text.replace("\'ve", " have")
    text = text.replace("can't", "can not")
    text = text.replace("n't", " not")
    text = text.replace("i'm", "i am")
    text = text.replace("'re", " are")
    text = text.replace("\'d", " would")
    text = text.replace("\'ll", " will")
    text = text.replace("\n", " ")
    for c in string.punctuation:
        text = text.replace(c, "")

    return text

In [87]:
data_train['comments_cleaned'] = data_train["comment_text"].apply(preprocess_text)

In [88]:
data_train

,comment_text,toxic,text_clean,comments_cleaned
22689,"""::No. The reason some excellent editors leav...",0,reason excellent editor leave reasoned fully e...,no the reason some excellent editors leave is...
154371,Comment Will it merge back into Historicity of...,0,comment merge back historicity jesus tiny part...,comment will it merge back into historicity of...
93418,Well then I might do that if I don't get the l...,0,well might dont get list done time although st...,well then i might do that if i do not get the ...
31049,"Several years later, I finally add something t...",0,several year later finally add something talk ...,several years later i finally add something to...
47906,Could you please specify the licence of the ph...,0,could please specify licence photo download se...,could you please specify the licence of the ph...
...,...,...,...,...
153719,In the meantime I can't even edit User talk:Ro...,0,meantime cant even edit user talkrompecorporat...,in the meantime i can not even edit user talkr...
148645,""" April 2011 (UTC)\nI don't dispute that the i...",0,april 2011 utc dont dispute image add article ...,april 2011 utc i do not dispute that the imag...
113159,I agree that this article is a WP:NEOLOGISM vi...,0,agree article wpneologism violation two source...,i agree that this article is a wpneologism vio...
104771,""" These comments alone constitute evidence tha...",0,comment alone constitute evidence mediation ka...,these comments alone constitute evidence that...


In [89]:
data_train[data_train.comments_cleaned == 'some jerkoff blocked me']

,comment_text,toxic,text_clean,comments_cleaned


In [90]:
# ignore severe_toxic, obscene, threat, insult, identity_hate
data_train_transformed = data_train.loc[((data_train.severe_toxic==0) & (data_train.obscene==0) 
                                    & (data_train.threat == 0) & (data_train.insult == 0) 
                                       & (data_train.identity_hate == 0))]
y = data_train_transformed[['toxic']]

AttributeError: 'DataFrame' object has no attribute 'severe_toxic'

In [91]:
data_train[data_train.toxic == 1]

,comment_text,toxic,text_clean,comments_cleaned
125996,FUCK U ALL \n\nFUCK U ALL,1,fuck u fuck u,fuck u all fuck u all
111470,Mud \n\nWhat the fuck are you doing? I added ...,1,mud fuck added adjusted score mud dont undo ch...,mud what the fuck are you doing i added the...
16367,"""\n""""Vandalism""""? Sorry, mister, but I haven't...",1,vandalism sorry mister havent conducted vandal...,vandalism sorry mister but i have not conduct...
92963,I am coming to get you \n\ni know where you li...,1,coming get know live im going come rape,i am coming to get you i know where you live...
45928,If that is vandalism then any accusation of be...,1,vandalism accusation sockpuppet vandalism sock...,if that is vandalism then any accusation of be...
...,...,...,...,...
59646,If you have no interest in the whole story the...,1,interest whole story get f,if you have no interest in the whole story the...
59711,"""\n\n Fuck you \n\nRe: """"mark williams radio h...",1,fuck mark williams radio host ground zero mosq...,fuck you re mark williams radio host and ...
17384,ARE IGNORANT PENDEJO IS 185 IS NAME IS CONFIRM...,1,ignorant pendejo 185 name confirmed ok,are ignorant pendejo is 185 is name is confirm...
121173,You idiots always demand I explain myself on t...,1,idiot always demand explain talk page always d...,you idiots always demand i explain myself on t...


In [92]:
import csv
data_train['label'] = ['__label__'+str(s) for s in data_train['toxic']]
data = data_train[['comments_cleaned', 'label']]
data.to_csv('comments.txt', index=False, sep=' ', header=False,quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [93]:
data_train[['comments_cleaned', 'label']]
data_train = data_train.fillna('')

In [94]:
import fasttext

model = fasttext.train_supervised(input="comments2.txt")

In [95]:
def get_prediction(fast_out):
    return int(fast_out[0][0].split('__label__')[1])

In [98]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

##Training accuracy
preds = [get_prediction(model.predict(x)) for x in data_train['comments_cleaned']]
print('accuracy:', accuracy_score(data_train['toxic'], preds),'\n')

##Training F1-score
print('F1-score:', f1_score(data_train['toxic'], preds),'\n')

accuracy: 0.9333924207020654 

F1-score: 0.4962762356127285 



In [99]:
from sklearn.metrics import classification_report

print(classification_report(data_train['toxic'], preds))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96    100993
           1       0.90      0.34      0.50     10706

    accuracy                           0.93    111699
   macro avg       0.92      0.67      0.73    111699
weighted avg       0.93      0.93      0.92    111699



In [23]:
model.save_model("model_toxic.bin")

In [28]:
model.predict("some jerkoff blocked me")

(('__label__1',), array([1.00001001]))

In [12]:
# ignore severe_toxic, obscene, threat, insult, identity_hate
toxic_non_toxic_data = data_train_transformed.loc[((data_train_transformed.severe_toxic==0) & (data_train_transformed.obscene==0) 
                                    & (data_train_transformed.threat == 0) & (data_train_transformed.insult == 0) 
                                       & (data_train_transformed.identity_hate == 0))]
y = toxic_non_toxic_data[['toxic']]

In [38]:
data_df = pd.read_pickle("Data/train_data.pkl")

In [39]:
toxic_data['comments_cleaned'] = data_df["comment_text"].apply(preprocess_text)

In [40]:
toxic_data[toxic_data.toxic == 1]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comments_cleaned
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,cocksucker before you piss around on my work
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0,hey what is it talk what is it an exclusive...
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0,bye do not look come or think of comming bac...
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1,you are gay or antisemmitian archangel white...
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0,NaN
...,...,...,...,...,...,...,...,...,...
159494,fef4cf7ba0012866,"""\n\n our previous conversation \n\nyou fuckin...",1,0,1,0,1,1,NaN
159514,ff39a2895fc3b40e,YOU ARE A MISCHIEVIOUS PUBIC HAIR,1,0,0,0,1,0,you are a mischievious pubic hair
159541,ffa33d3122b599d6,Your absurd edits \n\nYour absurd edits on gre...,1,0,1,0,1,0,your absurd edits your absurd edits on great...
159546,ffb47123b2d82762,"""\n\nHey listen don't you ever!!!! Delete my e...",1,0,0,0,1,0,NaN


In [41]:
toxic_comments = toxic_data.loc[toxic_data.toxic == 1]

non_toxic_comments = toxic_data.loc[((toxic_data.toxic == 0) & (toxic_data.severe_toxic==0) & (toxic_data.obscene==0) 
                                    & (toxic_data.threat == 0) & (toxic_data.insult == 0) & (toxic_data.identity_hate == 0))]

In [42]:
# ignore severe_toxic, obscene, threat, insult, identity_hate
toxic_non_toxic_data = toxic_data.loc[((toxic_data.severe_toxic==0) & (toxic_data.obscene==0) 
                                    & (toxic_data.threat == 0) & (toxic_data.insult == 0) 
                                       & (toxic_data.identity_hate == 0))]
y = toxic_non_toxic_data['toxic']

In [43]:
toxic_non_toxic_data['toxic']

0         0
1         0
2         0
3         0
4         0
         ..
159566    0
159567    0
159568    0
159569    0
159570    0
Name: toxic, Length: 149012, dtype: int64

In [44]:
import csv
toxic_non_toxic_data['label'] = ['__label__'+str(s) for s in toxic_non_toxic_data['toxic']]
data = toxic_non_toxic_data[['comments_cleaned', 'label']]
data.to_csv('comments2.txt', index=False, sep=' ', header=False,quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [67]:
toxic_non_toxic_data[['comments_cleaned', 'label']]
toxic_non_toxic_data = toxic_non_toxic_data.fillna('')

In [68]:
import fasttext

model = fasttext.train_supervised(input="comments2.txt")

In [48]:
def get_prediction(fast_out):
    return int(fast_out[0][0].split('__label__')[1])

In [69]:
get_prediction(model.predict('explanation why the edits made under my username hardcore metallica fan were reverted they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now892053827'))
#model.predict('cocksucker before you piss around on my work')

0

In [70]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

##Training accuracy
preds = [get_prediction(model.predict(x)) for x in toxic_non_toxic_data['comments_cleaned']]
print('accuracy:', accuracy_score(toxic_non_toxic_data['toxic'], preds),'\n')

##Training F1-score
print('F1-score:', f1_score(toxic_non_toxic_data['toxic'], preds),'\n')

accuracy: 0.9642981773280004 

F1-score: 0.2109166419460101 



0         explanation why the edits made under my userna...
1         daww he matches this background colour i am se...
2         hey man i am really not trying to edit war its...
3          more i can not make any real suggestions on i...
4         you sir are my hero any chance you remember wh...
                                ...                        
159566    and for the second time of asking when your vi...
159567    you should be ashamed of yourself   that is a ...
159568    spitzer   umm theres no actual article for pro...
159569    and it looks like it was actually you who put ...
159570     and  i really do not think you understand  i ...
Name: comments_cleaned, Length: 149012, dtype: object

0         0
1         0
2         0
3         0
4         0
         ..
159566    0
159567    0
159568    0
159569    0
159570    0
Name: toxic, Length: 149012, dtype: int64

In [73]:
from sklearn.metrics import classification_report

print(classification_report(toxic_non_toxic_data['toxic'], preds))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98    143346
           1       0.66      0.13      0.21      5666

    accuracy                           0.96    149012
   macro avg       0.81      0.56      0.60    149012
weighted avg       0.95      0.96      0.95    149012

